In [1]:
import os
from datetime import datetime
from pydantic import BaseModel
from sqlalchemy import ForeignKey, create_engine, URL, select
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

from dotenv import load_dotenv


In [44]:
url_object = URL.create(
    "postgresql+psycopg",
    username=os.getenv("SJSUBARK_PSQL_USER"),
    password=os.getenv("SJSUBARK_PSQL_PASSWORD"),
    host=os.getenv("SJSUBARK_PSQL_HOST"),
    database=os.getenv("SJSUBARK_PSQL_DB"),
    port=int(os.getenv("SJSUBARK_PSQL_PORT", 5432)),
)
db = create_engine(url_object)


#loads in garage_fullness + calendar
init_query = '''
SELECT gf.utc_timestamp, gf.fullness, gf.garage_id, c.is_weekend, c.is_campus_closed 
FROM garage_fullness gf JOIN calendar c  
ON gf.utc_timestamp >= c.utc_start
   AND gf.utc_timestamp < c.utc_end
;'''
garage_data = pd.read_sql(init_query, db)
garage_data.sort_values(["utc_timestamp", "garage_id"])

,utc_timestamp,fullness,garage_id,is_weekend,is_campus_closed
4,2025-08-26 04:10:01.000000,40,1,False,False
3022,2025-08-26 04:10:01.000000,15,2,False,False
3021,2025-08-26 04:10:01.000000,24,3,False,False
3227,2025-08-26 04:10:01.000000,1,4,False,False
3228,2025-08-26 04:20:01.000000,38,1,False,False
...,...,...,...,...,...
2828,2026-01-17 02:10:01.639075,0,4,False,False
2830,2026-01-17 02:20:01.890031,44,1,False,False
2831,2026-01-17 02:20:01.890031,3,2,False,False
2832,2026-01-17 02:20:01.890031,16,3,False,False


In [45]:
import numpy as np
import pandas as pd
from datetime import datetime

In [48]:
#Remove Non-Interval Data, AKA "if two records of the same garage is not 10 minutes apart, ignore it"

#CONFIGURATION
NUM_OF_GARAGE = 4
INTERVAL_OFFSET_MIN = 1
#######

result = []

for garage_id in range(1, NUM_OF_GARAGE + 1):
    g = garage_data.loc[garage_data['garage_id'] == garage_id].copy()
    g['lag1'] = g['fullness'].shift(1)
    g['time_lag1_diff'] = g['utc_timestamp'] - g['utc_timestamp'].shift(1)
    diff = g.loc[(g['time_lag1_diff'] < pd.Timedelta(10 + INTERVAL_OFFSET_MIN, unit="m")) & (g['time_lag1_diff'] > pd.Timedelta(10 - INTERVAL_OFFSET_MIN, unit="m"))]
    result.append(diff)

garage_data_cleaned = pd.concat(result, ignore_index=True)
garage_data_cleaned.drop('time_lag1_diff', axis=1)
garage_data_cleaned['is_weekend'] = garage_data_cleaned['is_weekend'].astype(int)
garage_data_cleaned['is_campus_closed'] = garage_data_cleaned['is_campus_closed'].astype(int)
garage_data_cleaned

,utc_timestamp,fullness,garage_id,is_weekend,is_campus_closed,lag1,time_lag1_diff
0,2026-01-13 04:20:02.121816,44,1,0,0,44.0,0 days 00:10:00.266216
1,2026-01-13 04:40:01.597671,44,1,0,0,44.0,0 days 00:10:00.223037
2,2026-01-13 05:00:02.159096,44,1,0,0,44.0,0 days 00:10:00.293478
3,2026-01-13 05:20:01.912403,44,1,0,0,44.0,0 days 00:10:00.481949
4,2026-01-13 05:40:01.505449,44,1,0,0,44.0,0 days 00:09:59.302015
...,...,...,...,...,...,...,...
77460,2026-01-12 22:30:01.614022,0,4,0,0,0.0,0 days 00:10:00.255632
77461,2026-01-13 00:30:01.889069,0,4,0,0,0.0,0 days 00:10:00.228887
77462,2026-01-13 00:40:02.125907,0,4,0,0,0.0,0 days 00:10:00.236838
77463,2026-01-13 00:50:01.533205,0,4,0,0,0.0,0 days 00:09:59.407298


In [54]:
garage_data_cleaned['utc_timestamp'] = pd.to_datetime(garage_data_cleaned['utc_timestamp'], utc=True)
garage_data_cleaned['hour'] = garage_data_cleaned['utc_timestamp'].dt.tz_convert('America/Los_Angeles').dt.hour
garage_data_cleaned['day_of_week'] = garage_data_cleaned['utc_timestamp'].dt.dayofweek
garage_data_cleaned

,utc_timestamp,fullness,garage_id,is_weekend,is_campus_closed,lag1,time_lag1_diff,hour,day_of_week
0,2026-01-13 04:20:02.121816+00:00,44,1,0,0,44.0,0 days 00:10:00.266216,20,1
1,2026-01-13 04:40:01.597671+00:00,44,1,0,0,44.0,0 days 00:10:00.223037,20,1
2,2026-01-13 05:00:02.159096+00:00,44,1,0,0,44.0,0 days 00:10:00.293478,21,1
3,2026-01-13 05:20:01.912403+00:00,44,1,0,0,44.0,0 days 00:10:00.481949,21,1
4,2026-01-13 05:40:01.505449+00:00,44,1,0,0,44.0,0 days 00:09:59.302015,21,1
...,...,...,...,...,...,...,...,...,...
77460,2026-01-12 22:30:01.614022+00:00,0,4,0,0,0.0,0 days 00:10:00.255632,14,0
77461,2026-01-13 00:30:01.889069+00:00,0,4,0,0,0.0,0 days 00:10:00.228887,16,1
77462,2026-01-13 00:40:02.125907+00:00,0,4,0,0,0.0,0 days 00:10:00.236838,16,1
77463,2026-01-13 00:50:01.533205+00:00,0,4,0,0,0.0,0 days 00:09:59.407298,16,1
